# langchain-parallel Demo: AI Agent with Web Research Tools

This notebook demonstrates how to build an AI agent that can search the web, extract content from URLs, and answer questions with real-time information using **langchain-parallel**.

We'll showcase three powerful tools:
- **ParallelWebSearchTool**: Search the web with natural language queries
- **ParallelExtractTool**: Extract clean content from web pages
- **ChatParallelWeb**: Chat model with built-in web research capabilities

Then we'll combine them into a **LangGraph agent** that can autonomously decide which tools to use to answer complex questions.

## Setup

First, let's install the required packages:

In [20]:
%pip install --quiet -U "langchain>=1.1.0" langchain-anthropic "langchain-parallel>=0.2.0"

Note: you may need to restart the kernel to use updated packages.


### Credentials

Set up your API keys. You'll need:
- **Parallel API key**: Get one at [parallel.ai](https://parallel.ai)
- **Anthropic API key**: For the agent's LLM (Claude)

In [21]:
import getpass
import os

if not os.environ.get("PARALLEL_API_KEY"):
    os.environ["PARALLEL_API_KEY"] = getpass.getpass("Parallel API key: ")

if not os.environ.get("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API key: ")

## The Tools

Let's initialize our three tools and see them in action.

In [22]:
from langchain_parallel import ParallelWebSearchTool, ParallelExtractTool, ChatParallelWeb

# Initialize the tools
search_tool = ParallelWebSearchTool()
extract_tool = ParallelExtractTool()
chat = ChatParallelWeb(model="speed")

print(f"Search Tool: {search_tool.name}")
print(f"Extract Tool: {extract_tool.name}")
print(f"Chat Tool: {chat.get_name()}")

Search Tool: parallel_web_search
Extract Tool: parallel_extract
Chat Tool: ChatParallelWeb


### 1. ParallelWebSearchTool

Search the web using natural language. Perfect for finding current information, news, and research.

In [23]:
# Search using a natural language objective
search_result = search_tool.invoke({
    "objective": "What are the latest breakthroughs in AI agents and autonomous systems?",
    "max_results": 5
})

print(f"Found {len(search_result['results'])} results:\n")
for i, result in enumerate(search_result['results'], 1):
    print(f"{i}. {result['title']}")
    print(f"   {result['url']}\n")

Found 5 results:

1. The Latest AI News and AI Breakthroughs that Matter Most
   https://www.crescendo.ai/news/latest-ai-news-and-updates

2. Autonomous AI Agents: 7 Breakthroughs Reshaping Our Future
   https://www.cognitivetoday.com/2025/12/autonomous-ai-agents-breakthroughs/

3. autonomous systems News & Articles - IEEE Spectrum
   https://spectrum.ieee.org/tag/autonomous-systems

4. AI Agents Lead The 8 Tech Trends Transforming ...
   https://www.forbes.com/sites/bernardmarr/2025/12/01/ai-agents-lead-the-8-tech-trends-transforming-enterprise-in-2026/

5. AI Agents in 2025: Expectations vs. Reality
   https://www.ibm.com/think/insights/ai-agents-2025-expectations-vs-reality



### 2. ParallelExtractTool

Extract clean, structured content from any URL. Great for reading articles, documentation, or research papers.

In [24]:
# Extract content from a URL
extract_result = extract_tool.invoke({
    "urls": ["https://en.wikipedia.org/wiki/Large_language_model"]
})

print(f"Title: {extract_result[0]['title']}")
print(f"Content length: {len(extract_result[0]['content'])} characters")
print(f"\nPreview:\n{extract_result[0]['content'][7000:8500]}...")

Title: Large language model - Wikipedia
Content length: 196689 characters

Preview:
asks, especially [language generation](/wiki/Natural_language_generation "Natural language generation") . <sup>[[ 1 ]]()</sup> <sup>[[ 2 ]]()</sup> The largest and most capable LLMs are [generative](/wiki/Generative_artificial_intelligence "Generative artificial intelligence") pre-trained [transformers](/wiki/Transformer_architecture "Transformer architecture") ( [GPTs](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer") ) and provide the core capabilities of modern [chatbots](/wiki/Chatbot "Chatbot") . LLMs can be [fine-tuned](/wiki/Fine-tuning_\(deep_learning\) "Fine-tuning (deep learning)") for specific tasks or guided by [prompt engineering](/wiki/Prompt_engineering "Prompt engineering") . <sup>[[ 3 ]]()</sup> These models acquire [predictive power](/wiki/Predictive_learning "Predictive learning") regarding [syntax](/wiki/Syntax "Syntax") , [semantics](/wiki/Semantics "Sem

### 3. ChatParallelWeb

A chat model with built-in web research capabilities. It can access real-time information directly.

In [25]:
response = chat.invoke([
    ("system", "You are a helpful assistant with access to real-time web information."),
    ("human", "What are the top AI research news stories today?")
])

print(response.content)

Here are some of the top AI research news stories from today, December 1, 2025, based on the provided search results:

**AI Model and Technology Advancements:**

*   **Runway's Gen 4.5 AI Video Model:** Runway announced Gen 4.5, a new AI model for generating high-definition videos from written prompts. It is reported to outperform models from Google and OpenAI in an independent benchmark, specifically in understanding physics, human motion, camera movements, and cause and effect.
*   **TwelveLabs' Marengo 3.0:** TwelveLabs launched Marengo 3.0, a video foundation model available on Amazon Bedrock, that understands video in a human-like manner by connecting dialogue, gestures, movement, and emotion across time.
*   **MIT Research:** MIT researchers have discovered a shortcoming that makes large language models less reliable. They also debuted a generative AI model, BoltzGen, that can create molecules for hard-to-treat diseases.
*   **Purdue University's RAPTOR:** Purdue University resea

## Building an Agent with LangChain 1.x

Now let's combine these tools into a powerful agent using LangChain's `create_agent`. The agent can autonomously decide when to search the web, when to extract content from URLs, and how to synthesize information to answer complex questions.

This uses the new LangChain 1.x API which provides:
- Simple model specification via string (e.g., `"anthropic:claude-opus-4-5-20251101"`)
- `system_prompt` for guiding agent behavior
- Built on LangGraph's durable runtime under the hood

In [26]:
from langchain.agents import create_agent

# Create the agent with our tools (using Claude Opus 4.5)
tools = [search_tool, extract_tool]

agent = create_agent(
    "anthropic:claude-opus-4-5-20251101",
    tools,
    system_prompt="""You are a helpful research assistant with access to web search and content extraction tools.

When answering questions:
1. Use parallel_web_search to find relevant, current information
2. Use parallel_extract to get detailed content from specific URLs when needed
3. Synthesize the information into a clear, comprehensive answer
4. Always cite your sources with URLs

Be thorough but concise in your responses."""
)

### Agent in Action

Let's ask our agent some questions that require real-time web research!

In [27]:
import textwrap

# Helper function to run the agent with streaming output
def ask_agent(question: str, width: int = 140):
    print(f"Question: {question}\n")
    print("-" * width)
    
    for event in agent.stream({"messages": [("human", question)]}, stream_mode="values"):
        message = event["messages"][-1]
        if hasattr(message, 'content') and message.content:
            if hasattr(message, 'tool_calls') and message.tool_calls:
                for tool_call in message.tool_calls:
                    print(f"🔧 Using tool: {tool_call['name']}")
            elif message.type == "ai":
                wrapped = textwrap.fill(message.content, width=width)
                print(f"\n💬 Answer:\n{wrapped}")
    
    return event

In [28]:
result = ask_agent("What are the most significant AI developments this week?")

Question: What are the most significant AI developments this week?

--------------------------------------------------------------------------------------------------------------------------------------------
🔧 Using tool: parallel_web_search
🔧 Using tool: parallel_web_search

💬 Answer:
Based on my research, here are the **most significant AI developments this week** (late November/early December 2025):  ## 🎬 Runway Launches
Gen 4.5 - Leading AI Video Model **Runway** announced **Gen 4.5**, their new AI video generation model that now **ranks #1 on the Video
Arena leaderboard**, surpassing Google's Veo 3 and OpenAI's Sora 2 Pro. The model excels at understanding physics, human motion, and camera
movements. CEO Cristóbal Valenzuela noted they "managed to out-compete trillion-dollar companies with a team of 100 people." - Source:
[CNBC](https://www.cnbc.com/2025/12/01/runway-gen-4-5-video-model-google-open-ai.html)  ## ☁️ AWS re:Invent 2025 Major Announcements
**Amazon Web Services** is 

In [29]:
result = ask_agent("Give me a one sentence summary of https://docs.langchain.com/oss/python/integrations/tools/parallel_search")

Question: Give me a one sentence summary of https://docs.langchain.com/oss/python/integrations/tools/parallel_search

--------------------------------------------------------------------------------------------------------------------------------------------
🔧 Using tool: parallel_extract

💬 Answer:
The `ParallelWebSearchTool` is a LangChain integration that provides access to Parallel's real-time web search API, combining search,
scraping, and content extraction into a single API call that returns structured, LLM-optimized results with features like domain filtering,
customizable excerpts, and async support.
